In [1]:
import pandas as pd
import numpy as np
import re
from underthesea import word_tokenize
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [2]:
# Nhập dữ liệu
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]


In [3]:
# Gắn nhãn cho bộ dữ liệu
def label_sentiment(rating):
    if rating in [1, 2]:
        return '0'
    elif rating == 3:
        return '1'
    elif rating in [4, 5]:
        return '2'
    else:
        return '3'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
df = df.dropna(subset=['Comment'])

# Kiểm tra và loại bỏ dữ liệu trùng lặp
df = df.drop_duplicates(['Comment'])

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)

In [6]:
# Xây dựng hàm token từ underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

vectorizer = TfidfVectorizer(tokenizer=tokenize_and_build_vocab_vietnamese, token_pattern=None,
                              max_features=5000, ngram_range=(1, 2), max_df=0.85, min_df=5)

# Fit and transform the data
X_tfidf = vectorizer.fit_transform(df['Comment'].values.tolist())

In [7]:
# Mã hóa cho cột nhãn
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Sentiment'].values.tolist())

In [8]:
# Cân bằng dữ liệu
print('Trước khi cân bằng:', Counter(y))
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)
print('Sau khi cân bằng:', Counter(y_resampled))

Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [9]:
# Chia tập dữ liệu thành tập huấn luyện và tập validation (validation + test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=42)

# Convert sparse matrix to array
X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

In [10]:
# Build the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
# Train the model
epochs = 50  # Tăng số epochs
batch_size = 32  # Giảm kích thước batch

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001)])


Epoch 1/50
47/47 [==============================] - 3s 20ms/step - loss: 1.0452 - accuracy: 0.5718 - val_loss: 0.9138 - val_accuracy: 0.7387
Epoch 2/50
47/47 [==============================] - 0s 8ms/step - loss: 0.6522 - accuracy: 0.8243 - val_loss: 0.4619 - val_accuracy: 0.8560
Epoch 3/50
47/47 [==============================] - 0s 7ms/step - loss: 0.3006 - accuracy: 0.9078 - val_loss: 0.3157 - val_accuracy: 0.8933
Epoch 4/50
47/47 [==============================] - 0s 7ms/step - loss: 0.1545 - accuracy: 0.9613 - val_loss: 0.3033 - val_accuracy: 0.9040
Epoch 5/50
47/47 [==============================] - 0s 7ms/step - loss: 0.0866 - accuracy: 0.9826 - val_loss: 0.2866 - val_accuracy: 0.9093
Epoch 6/50
47/47 [==============================] - 1s 12ms/step - loss: 0.0505 - accuracy: 0.9933 - val_loss: 0.3017 - val_accuracy: 0.9013
Epoch 7/50
47/47 [==============================] - 1s 12ms/step - loss: 0.0260 - accuracy: 0.9973 - val_loss: 0.3277 - val_accuracy: 0.8987
Epoch 8/50
47/47 

In [12]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

20/20 [==============================] - 0s 5ms/step - loss: 0.2932 - accuracy: 0.9151
Test Score: 0.29316678643226624
Test Accuracy: 0.9150640964508057


In [13]:
# Báo cáo đánh giá mô hình
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

20/20 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       209
           1       0.84      0.96      0.90       204
           2       0.97      0.87      0.92       211

    accuracy                           0.92       624
   macro avg       0.92      0.92      0.92       624
weighted avg       0.92      0.92      0.92       624



In [14]:
# Đánh giá mới cần kiểm thử
new_review = "Shop có cho đổi điện thoại không?"

# Tiền xử lý đánh giá mới
new_review = normalize_text(new_review)

# Chuyển đổi đánh giá mới sang TF-IDF
new_review_tfidf = vectorizer.transform([new_review]).toarray()

# Dự đoán nhãn cho đánh giá mới
prediction = model.predict(new_review_tfidf)
predicted_label = np.argmax(prediction, axis=1)

# Chuyển đổi nhãn dự đoán từ số sang tên
predicted_sentiment = label_encoder.inverse_transform(predicted_label)
print(f"Đánh giá: {new_review}")
print(f"Dự đoán cảm xúc: {predicted_sentiment[0]}")

1/1 [==============================] - 0s 340ms/step
Đánh giá: shop có cho đổi điện thoại không
Dự đoán cảm xúc: 1
